In [ ]:
#include <dune/common/parallel/mpihelper.hh>

// Initialize (fake) mpi
auto& mpi_helper = Dune::MPIHelper::instance(int{}, (char**){});
auto comm = mpi_helper.getCollectiveCommunication();

In [ ]:
#include <dune/logging/logging.hh>

// Initialize loggers
Dune::Logging::Logging::init(comm, {});

In [ ]:
#include <dune/grid/io/file/gmshreader.hh>
#include <dune/grid/uggrid.hh>

// Load 2D grid from gmsh file
constexpr int dim = 2;
using Grid = Dune::UGGrid<dim>;
std::shared_ptr<Grid> grid_ptr = Dune::GmshReader<Grid>::read("grids/circle.msh");

In [ ]:
#include <dune/common/parametertree.hh>
#include <dune/common/parametertreeparser.hh>

// Setup a model for diffusion-reaction systems
//   We pass a string with INI format that describes
//   the (multiple) compartment properties

std::istringstream config_stream{
    // "inside" compartment with two variables (u,v)
          "inside.diffusion.u  = 1e-3           \n"
          "inside.diffusion.v  = 1e-3           \n"
          "inside.reaction.u = u*v              \n"
          "inside.reaction.v = u*v              \n"
          "inside.initial.u = sin(x) + cos(y)   \n"
          "inside.initial.v = 0                 \n"
    // "outside" compartment with one variable (u)
          "outside.diffusion.u = 1e-3   \n"
          "outside.reaction.u  = 1e-3   \n"
          "outside.initial.u = 1        \n"
};

// create the config file
Dune::ParameterTree config = Dune::ParameterTreeParser::readINITree(config_stream);

In [ ]:
#include <dune/copasi/model/diffusion_reaction.hh>

// Make a model that interprets our configuration
Dune::Copasi::DiffusionReactionModel model{config};

// Get state for the grid with properties of the "inside" compartment
auto state = model.make_singledomain_multicomponent_state(grid_ptr, "inside");

In [ ]:
// Interpolate analytical expressions into the state
const Dune::ParameterTree& initial_config = config.sub("inside.initial");

// Interpret initial value expressions into a grid function...
auto grid_function = Dune::Copasi::make_multicomponent_grid_function(initial_config, state.space().gridView());

// ...and interpolate it to the current state
model.interpolate(state, grid_function);

In [ ]:
// add a default writer to the state
state.writer = model.make_vtk_writer();

// write current state into a file named inside and override old files
state.write("inside", true);

In [ ]:
#include <dune/copasi/common/stepper.hh>

Dune::ParameterTree timestep_config;

timestep_config["rk_method"] = "alexander_2";
timestep_config["decrease_factor"] = "0.5";
timestep_config["increase_factor"] = "1.5";
timestep_config["min_step"] = "1e-3";
timestep_config["max_step"] = "1e-1";

timestep_config["newton.reduction"] = "1e-8";
timestep_config["newton.min_linear_reduction"] = "1e-3";
timestep_config["newton.fixed_linear_reduction"] = "false";
timestep_config["newton.max_iterations"] = "40";
timestep_config["newton.absolute_limit"] = "1e-12";
timestep_config["newton.reassemble_threshold"] = "0.0";
timestep_config["newton.keep_matrix"] = "true";
timestep_config["newton.force_iteration"] = "false";

timestep_config["newton.linear_search.strategy"] = "hackbuschReusken";
timestep_config["newton.linear_search.max_iterations"] = "10";
timestep_config["newton.linear_search.damping_factor"] = "0.5";

auto stepper = Dune::Copasi::make_default_stepper(timestep_config);

double end_time = 10;
double initial_step = 0.1;
auto at_end_of_step = [](auto& state){/*state.write("inside", true);*/};
stepper.evolve(model, state, state, initial_step, end_time, at_end_of_step);

In [ ]:
// write current state into a file named inside and override old files
state.write("inside", true);